<span style='color:red'> NOTE: You can only pass the lab, when you provide both code and markdown </span>

Use Code for your analysis
Use Markdown to document and elaborate on your findings, conclusions, assertions, etc.

# DS_ML_I_P5: Classification of Image Data

## 5.1 Load the MNIST Data-Set
* get the data using code below
* assemble information on number of images, size of images, distribution among different classes

In [ ]:
from sklearn.datasets import fetch_openml
from math import sqrt
import pandas as pd
import numpy as np
# if we loaded the dataset using fetch_openml before, we can use the previously stored parquet file (which laods much faster)
preloaded = True

In [ ]:
if preloaded == False:
    X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
    display(type(X), type(y))
else:
    %time df_mnist = pd.read_parquet("MNIST.parquet")
    X = df_mnist.iloc[:, 0:784]
    y = df_mnist.iloc[:, 784]

## 5.2 Visualize the input data
* Display at least two images for each class in the dataset
* Show the class identity per image

## 5.3 Train a k-NN Classifier and Check the Performance

### 5.3.1 Import the required modules for classifier and performance measurement

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

### 5.3.2 Train the classifier using 3-fold cross_validation 
* create a training and testing set
* train the classifer with cross_validate, check the documentation: What does cross_validate return?

### 5.3.3 Use the best classifier from the previuous run and check with your test set 
* Generate a classification report
* Generate a confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# 5.4 Train a Decision Tree Classifier and Check the Performance

In [ ]:
from sklearn.tree import DecisionTreeClassifier

### 5.4.1 Train the classifier using 3-fold cross_validation

### 5.4.2 Check the performance with your test set

## 5.5 Improve with random forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

## 5.6 Further improvement: Gradient Boosting (non Random) Forests

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier